# Harmony Browse Image Service (HyBIG) Example Notebook

This Jupyter notebook demonstrates the functionality of the Harmony Browse Image Service (HyBIG)

#### Contact
* **Slack:** #harmony-service-providers
* **JIRA:**  [SDPS Data Services](https://bugs.earthdata.nasa.gov/secure/RapidBoard.jspa?rapidView=757&view=planning.nodetail)


HyBIG converts GeoTIFF inputs to Global Imagery Browse Services ([GIBS](https://nasa-gibs.github.io/gibs-api-docs/)) compatible PNG or JPEG outputs with metadata. This includes, where necessary, conversion to a GIBS supported Coordinate Reference System (CRS) and preferred resolutions. User customizations, including `crs`, `scale_extents`, and dimensions (via `height` & `width`), can be requested, but note the generated outputs may not be compatible with GIBS. 

Information on how HyBIG computes default values can be found in the [README.md](https://github.com/nasa/harmony-browse-image-generator/blob/main/README.md) file of this repository.

The examples use data from these collections:

  - MEaSUREs Vegetation Continuous Fields (VCF) Yearly Global 0.05 Deg V001 ([VCF5KYR](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1258119317-EEDTEST)) as GeoTIFF input data to demonstrate the HyBIG service against RGB color banded input. Each granule covers the full Earth.

  - ASTER Global Digital Elevation Model (GDEM) Version 3 ([ASTGTM](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1256584478-EEDTEST)) as GeoTIFF input data to demonstrate the HyBIG service using an input GeoTIFF with no color information. Each granule covers a 1° x 1° region at 1" resolution.
    
 
#### Authentication prerequisites:

The `harmony.Client` class will attempt to use credentials from a local `.netrc` file, located in the home directory of the filesystem where this notebook is running. This will need to contain entries for Earthdata Login for your UAT environment:

```
machine uat.urs.earthdata.nasa.gov
    login <uat_edl_username>
    password <uat_edl_password>
```

### Import required packages:

The cell below imports classes and functions from various Python packages, including:

* `harmony-py`: A package that allows for easy, interaction with the Harmony API that can be written directly in Python.
* `pathlib` : standard python package for working with object oriented paths.
* [`gdal`]: *optional* A geographic tranlator library that provides the `gdalinfo` binary needed for some cells to function. This can be easily installed with [conda](https://docs.conda.io/en/latest/).

In [ ]:
# creates an output directory for the downloaded files
from pathlib import Path

from harmony_service_lib import Client, Collection, Environment, Request

output_dir = Path('./hybig-output')
Path.mkdir(output_dir, exist_ok=True)

### Set up example variables:

This includes the Harmony `Client` object and `Collection` and `Granules` for each of the data sets to examine. 

In [ ]:
aster_collection = Collection(id='C1256584478-EEDTEST')
aster_granule = 'G1256584570-EEDTEST'

measures_collection = Collection(id='C1258119317-EEDTEST')
measures_granule = 'G1258119387-EEDTEST'

harmony_client = Client(env=Environment.UAT)

### Helper function to rename output images.

Harmony prepends a job identifier to processed images, to display them in this notebook the `rename_files` function will strip off the job id.

In [ ]:
def rename_files(file_paths):
    print('Renaming:')
    for file_path in file_paths:
        path = Path(file_path)
        filename = path.name
        new_filename = '_'.join(filename.split('_')[1:])
        new_path = path.parent / new_filename
        path.rename(new_path)
        print(new_path)

### Example: default request for MEaSUREs VCF5KYR 3-Band RGB GeoTIFF input

This makes a default request for the data without input parameters allowing the
HyBIG service to determine the defaults for a GIBS compatible output image.

The only selection made is the output image type, which can be either PNG or JPEG
and selection is done with the `format` keyword to the `Request`, by specifying
either 'image/png' or 'image/jpeg'


In [ ]:
# Valid values are 'image/png' and 'image/jpeg':
image_format = 'image/png'


# Specify a request to create a browse image from an MEaSUREs granule:
measures_request = Request(
    collection=measures_collection,
    granule_id=measures_granule,
    format=image_format,
)

# Submit the request and wait for it to complete:
measures_job_id = harmony_client.submit(measures_request)
harmony_client.wait_for_processing(measures_job_id, show_progress=True)

# Download all of the generated files:
example1_output_dir = output_dir / 'example1'
Path.mkdir(example1_output_dir, exist_ok=True)
downloaded_outputs = [
    file_future.result()
    for file_future in harmony_client.download_all(
        measures_job_id, overwrite=True, directory=example1_output_dir
    )
]

rename_files(downloaded_outputs)

The output PNG image:

<img src='hybig-output/example1/VCF5KYR_1991001_001_2018224205008.png' width='700px' align='left'>

In [ ]:
# view all of the output files
!ls "hybig-output/example1"

Three files are created and downloaded from Harmony:

A PNG image file: `VCF5KYR_1991001_001_2018224205008.png`
  - PNG files are the raster representation of the data in a standard image format.
  
An ESRI world file: `VCF5KYR_1991001_001_2018224205008.pgw`
  - World files establish an image-to-world transformation that converts the image coordinates to real-world coordinates.  This is the metadata file to submit to GIBS. *A drawback to these files is that they do not contain the Spatial Reference System (SRS) for the image, but only the GeoTransoform.*

An ARCGIS auxiliary file: `VCF5KYR_1991001_001_2018224205008.png.aux.xml`
  - An auxiliary file (.aux.xml) accompanies the raster in the same location and stores any additional information that cannot be stored in the raster file itself. In this case it contains the Spatial Reference System (SRS) and the GeoTransform describing the pixel location.  This file is used when importing the image into a GIS workspace.

The output image is a paletted PNG as that is GIBS preferred image format. A paletted PNG is an image with only one band of byte values and an associated color lookup table embedded in the image.  

This means that the data in the image is stored as values from 0 to 255 and each value maps to a color quartet of Red, Green, Blue and Alpha values. This reduces the size of the output image.

In the next cell, `gdalinfo` is run on the output PNG file. 
Notice there is only one band with color interpreted by Palette, and the actual color values are shown below the `Color Table`. 

In [ ]:
# if gdalinfo is installed you can see the palette associated with the PNG image.
!gdalinfo  hybig-output/example1/VCF5KYR_1991001_001_2018224205008.png

### Example:  specified spatial extents override GIBS-compatible defaults:

This example specifies a `scale_extent` in the request, which tells Harmony the spatial area of the output browse imagery. 

This example will use an ASTER granule, with the following extent:

* 22 ≤ longitude (degrees east) ≤ 23.2
* 0 ≤ latitude (degrees north) ≤ 1.2

The expected output should be a single tile, with a .2 degree border at the top and right edges.

In [ ]:
# The extent of the image [xmin, ymin, xmax, ymax]
scale_extent = [22, 0, 23.2, 1.2]

# Specify a request to create a browse image from an ASTER granule specifying
# the scale extent of the image:
extent_request = Request(
    collection=aster_collection,
    granule_id=aster_granule,
    scale_extent=scale_extent,
    crs='EPSG:4326',
    format='image/jpeg',
)

# Submit the request and wait for it to complete:
extent_job_id = harmony_client.submit(extent_request)
harmony_client.wait_for_processing(extent_job_id, show_progress=True)

# download all generated files:
example2_output_dir = output_dir / 'example2'
Path.mkdir(example2_output_dir, exist_ok=True)
downloaded_extent_outputs = [
    file_future.result()
    for file_future in harmony_client.download_all(
        extent_job_id, overwrite=True, directory=example2_output_dir
    )
]
rename_files(downloaded_extent_outputs)

#### The resulting output JPEG
Notice the .2 degree border on the north and west edges.

<img src="hybig-output/example2/ASTGTMV003_N00E022_dem.jpg" width=500 align='left' style="border:3px solid gray" />


**The cell below shows the tile extents match the requested tile extents.**

* 22 ≤ longitude (degrees east) ≤ 23.2
* 0 ≤ latitude (degrees north) ≤ 1.2

In [ ]:
# If gdal is installed this will show you the corner points associated with the output files.
!gdalinfo hybig-output/example2/ASTGTMV003_N00E022_dem.jpg | grep -E "Left|Right"

### Example:  spatial sizes (resolutions) override GIBS-compatible defaults:

This example demonstrates how a `scale_size` in the request sets the resolution of the produced browse imagery.  This example specifies two custom resolutions (one in each dimension), that are not GIBS-compatible defaults.

The output is a single image with the requested resolutions.

The example specifies resolutions of 1° in longitude and 2° in latitude. Choosing a y-dimension scale size that is twice as large as the x-dimension scale size means the outputs will look squashed in the vertical direction, the output size will be 360 x 90 

In [ ]:
# Set scales to 1° longitude and 2° latitude
scale_sizes = [1.0, 2.0]

# Specify a request to create a browse image from an MEaSUREs granule
# with the desired scale_sizes:
scale_size_request = Request(
    collection=measures_collection,
    granule_id=measures_granule,
    scale_size=scale_sizes,
    crs='EPSG:4326',
    format='image/png',
)

# Submit the request and wait for it to complete:
scale_size_job_id = harmony_client.submit(scale_size_request)
harmony_client.wait_for_processing(scale_size_job_id, show_progress=True)

# download all generated files:
example3_output_dir = output_dir / 'example3'
Path.mkdir(example3_output_dir, exist_ok=True)
downloaded_scale_size_outputs = [
    file_future.result()
    for file_future in harmony_client.download_all(
        scale_size_job_id, overwrite=True, directory=example3_output_dir
    )
]
rename_files(downloaded_scale_size_outputs)

The image generated is shown:


<img src='hybig-output/example3/VCF5KYR_1991001_001_2018224205008.png' align='left'/>

### Example: dimensions override GIBS-compatible defaults:

This example specifies the output browse image size by customizing `height` and `width` in the request. 

We choose height and width to be equal in this example. 

Because the input has twice as many pixels in the x-direction, the output browse image will look squashed in the horizontal direction.

In [ ]:
# Specify a request to create a browse image from an MEaSUREs granule
# specify both height and width to be the same:
dimensions_request = Request(
    collection=measures_collection,
    granule_id=measures_granule,
    height=180,
    width=180,
    format='image/png',
)

# Submit the request and wait for it to complete:
dimensions_job_id = harmony_client.submit(dimensions_request)
harmony_client.wait_for_processing(dimensions_job_id, show_progress=True)

# download all generated files:
example4_output_dir = output_dir / 'example4'
Path.mkdir(example4_output_dir, exist_ok=True)
downloaded_dimensions_outputs = [
    file_future.result()
    for file_future in harmony_client.download_all(
        dimensions_job_id, overwrite=True, directory=example4_output_dir
    )
]
rename_files(downloaded_dimensions_outputs)

The resulting square image is:

<img src='hybig-output/example4/VCF5KYR_1991001_001_2018224205008.png' align='left' />

## Example: Tiling 

In the next examples, carefully choosing the scale extent along with scale size, demonstrates the tiling behavior in HyBIG. 

### Example: untiled maximum size output image 
Start with a square `scale_extent` over Australia, 42° on a side. 

Choose a `scale_size`s so that the expected output is precisely the maxium size of an untiled image. 

```
australia_extent = [112, -47, 154, -5]
scale_size_x = (154 - 112) / 8192
scale_size_y = (-5 - -47) / 8192
```

We used 8192 because HyBIG can create a single untiled image of up to 8,192 x 8,192 cells large.



In [ ]:
# Square extent over Australia 42° x 42°
australia_extent = [112, -47, 154, -5]


# cell size = width[height] / total cells
scale_size_x = (australia_extent[2] - australia_extent[0]) / 8192
scale_size_y = (australia_extent[3] - australia_extent[1]) / 8192


australia_scale_size = [scale_size_x, scale_size_y]


# Specify a request to create a browse image from an ASTER granule specifying
# scale_extent and scale_size to create a single tile:
untiled_request = Request(
    collection=measures_collection,
    granule_id=measures_granule,
    scale_extent=australia_extent,
    scale_size=australia_scale_size,
    crs='EPSG:4326',
    format='image/png',
)


# Submit the request and wait for it to complete:
untiled_job_id = harmony_client.submit(untiled_request)
harmony_client.wait_for_processing(untiled_job_id, show_progress=True)

# download all generated files:
example5_output_dir = output_dir / 'example5'
Path.mkdir(example5_output_dir, exist_ok=True)
downloaded_untiled_outputs = [
    file_future.result()
    for file_future in harmony_client.download_all(
        untiled_job_id, overwrite=True, directory=example5_output_dir
    )
]
rename_files(downloaded_untiled_outputs)

In [ ]:
# The output file has the expected dimensions 8192 x 8192
!file hybig-output/example5/VCF5KYR_1991001_001_2018224205008.png

the untiled output image:

<img src='hybig-output/example5/VCF5KYR_1991001_001_2018224205008.png' width=350 align='left'/>


### Example: tiled output images

Again, start with same square `scale_extent` over Australia, 42° on a side. 

Decrease the `scale_size`s so that the expected output exceeds the maxium size of an untiled image forcing tiled output. 

```
australia_extent = [112, -47, 154, -5]
scale_size_x = (154 - 112) / 8192 + 10
scale_size_y = (-5 - -47) / 8192 + 10
```

The total number of output cells have been extened by 10 in each direction, so that the output will be tiled into 4096 x 4096 images.


In [ ]:
# Square extent over Australia 42° x 42°
australia_extent = [112, -47, 154, -5]


# width / total cells
scale_size_x = (australia_extent[2] - australia_extent[0]) / (8192 + 10)
scale_size_y = (australia_extent[3] - australia_extent[1]) / (8192 + 10)


australia_scale_size = [scale_size_x, scale_size_y]


# Specify a request to create a browse image from an ASTER granule specifying
# scale_extent and scale_size to create tiled images:
tiled_request = Request(
    collection=measures_collection,
    granule_id=measures_granule,
    scale_extent=australia_extent,
    scale_size=australia_scale_size,
    crs='EPSG:4326',
    format='image/png',
)


# Submit the request and wait for it to complete:
tiled_job_id = harmony_client.submit(tiled_request)
harmony_client.wait_for_processing(tiled_job_id, show_progress=True)

# download all generated files:
example5_output_dir = output_dir / 'example5'
Path.mkdir(example5_output_dir, exist_ok=True)
downloaded_tiled_outputs = [
    file_future.result()
    for file_future in harmony_client.download_all(
        tiled_job_id, overwrite=True, directory=example5_output_dir
    )
]
rename_files(downloaded_tiled_outputs)

In [ ]:
# The tiles have expected dimensions 4096 x 4096:
!file hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r00c00.png

In [ ]:
# The edge tiles make up the remaining 10 grid cells
!file hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r00c02.png

Some of the tiled output images:


<table align="left">
<tr>
<td> <img src="hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r00c00.png" width=150 height=150 align="left" /> </td>
<td> <img src="hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r00c01.png" width=150 height=150 align="left" /> </td>
</tr>
<tr>
<td> <img src="hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r01c00.png" width=150 height=150 align="left" /> </td>
<td> <img src="hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r01c01.png" width=150 height=150 align="left" /> </td>
</tr>
<tr>
<td> <img src="hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r02c00.png" width=150 height=150 align="left" /> </td>
<td> <img src="hybig-output/example5/VCF5KYR_1991001_001_2018224205008.r02c01.png" width=150 height=150 align="left" /> </td>
</tr>
</table>



#### Clean up the output notebook directory

In [ ]:
from shutil import rmtree

rmtree(output_dir)